In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [7]:
cd Kaggle-House-Prices-Regression/

/Users/adityasingh/Downloads/Kaggle-House-Prices-Regression


In [8]:
df = pd.read_csv('train.csv')

In [1]:
df.describe()

NameError: name 'df' is not defined

In [ ]:
x = 5